# Context
This notebook drives the training process for different models.

In [1]:
# Set project's environment variables
import os
import sys
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../../project.env")
sys.path.append(os.environ["PYTHONPATH"])

import pickle

In [2]:
# Import project-wide and PH2 specific variables and functions
import superheader as sup
import TRAIN.architecture.archeader as archeader



Chosen class grouping: two-classes


Directory /Users/diego/Desktop/iteso/TOG/ exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/src exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/bin exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/media exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/scores exists. Continuing with execution


# Datasets

In [3]:
sup.report_dir_if_not_exists(sup.PH1_DATA_ROOT)
sup.report_dir_if_not_exists(sup.PH2_DATA_ROOT)
sup.report_dir_if_not_exists(sup.PH3_DATA_ROOT)

Directory /Users/diego/Desktop/iteso/TOG/data/PH1/two-classes exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH2/two-classes exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH3/two-classes exists. Continuing with execution


In [4]:
import pandas as pd

In [5]:
data_config = {
  "PH2" : True,
  "PH3" : False,
  "reducer" : None,
  "kernel" : None,
  "n" : -1,
  "data_unit" : sup.DATA_S_PV,
  "label_col" : sup.class_numeric_column,
  "class_list" : "two-classes",
  "test_ratio" : 0.2
}

In [6]:
baseArch = archeader.Arch(data_config=data_config, df=None, model_path_dir=None)

/Users/diego/Desktop/iteso/TOG/src/TRAIN/architecture/archeader.py:93: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  spv_df = spf_df.groupby(sup.tag_columns+sup.class_columns).apply(self.__flatten_group).reset_index(drop=True)


In [7]:
df = baseArch.df
df


,class_numeric,f0_h_mean_x,f0_h_mean_y,f0_h_mean_z,f0_h_v1x,f0_h_v1y,f0_h_v1z,f0_h_v2x,f0_h_v2y,f0_h_v2z,...,f11_cp0z,f11_cp11x,f11_cp11y,f11_cp11z,f11_cp12x,f11_cp12y,f11_cp12z,f11_cp_h_mean_x,f11_cp_h_mean_y,f11_cp_h_mean_z
0,0,0.360239,0.416955,-0.006689,0.057899,-0.998047,-0.023436,-0.221157,-0.971920,-0.080376,...,-0.058999,-2.195826,0.758305,0.120381,-2.498878,1.796790,-0.575941,-3.106573,0.953093,-0.090265
1,1,0.351711,0.363593,-0.001278,0.128110,-0.991108,0.035963,-0.095874,-0.995393,-0.000389,...,-0.160177,-2.021350,0.773495,0.047998,-2.356061,1.763877,-0.672998,-2.971662,1.030349,-0.193626
2,0,0.496187,0.994735,-0.001383,0.507009,0.861901,-0.008269,0.198045,0.976251,-0.087821,...,-0.644612,-1.964399,1.739916,-0.185610,-1.559865,3.232175,-1.054860,-1.728090,2.489934,-0.456677
3,1,0.669912,0.352840,-0.012349,-0.140774,-0.987168,-0.075381,0.093798,-0.993471,-0.064945,...,-0.532817,-1.907883,1.487157,-0.082033,-1.745100,2.917575,-0.980489,-1.763566,2.255293,-0.365618
4,0,0.362565,0.390016,-0.006525,0.078281,-0.996684,-0.022218,-0.195734,-0.977112,-0.083306,...,0.004046,-2.093710,0.708905,0.165053,-2.465204,1.695280,-0.506559,-2.970079,0.907383,-0.015506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,1,0.638471,0.230422,-0.009267,-0.146881,-0.988618,-0.032563,0.171358,-0.981407,-0.086468,...,-0.019268,-0.874001,0.544017,0.017931,-1.077997,1.027761,-0.438979,-1.031278,0.685579,0.029543
209,0,0.360149,0.483891,-0.022221,0.113875,-0.978767,-0.170433,-0.286592,-0.900323,-0.327541,...,0.127829,-0.776137,0.577966,0.040399,-0.925767,0.882573,-0.288725,-1.376984,0.389957,0.091353
210,1,0.378590,0.251846,-0.011795,0.207972,-0.977056,-0.045919,-0.055048,-0.995213,-0.080751,...,0.083438,-0.599103,0.539396,-0.020511,-0.814500,0.781035,-0.309411,-1.245915,0.376465,0.060204
211,0,0.635323,0.273810,-0.007838,-0.153141,-0.986520,-0.057678,0.166022,-0.981470,-0.095678,...,-0.078067,-1.450643,0.842298,0.087333,-1.500647,1.807713,-0.559489,-1.482214,1.144109,0.018063


In [8]:
input_dim = 1044 // 12
input_dim

87

In [9]:
print(df.columns.to_list())

['class_numeric', 'f0_h_mean_x', 'f0_h_mean_y', 'f0_h_mean_z', 'f0_h_v1x', 'f0_h_v1y', 'f0_h_v1z', 'f0_h_v2x', 'f0_h_v2y', 'f0_h_v2z', 'f0_h_v3x', 'f0_h_v3y', 'f0_h_v3z', 'f0_wh0x', 'f0_wh0y', 'f0_wh0z', 'f0_wh1x', 'f0_wh1y', 'f0_wh1z', 'f0_wh2x', 'f0_wh2y', 'f0_wh2z', 'f0_wh3x', 'f0_wh3y', 'f0_wh3z', 'f0_wh4x', 'f0_wh4y', 'f0_wh4z', 'f0_wh5x', 'f0_wh5y', 'f0_wh5z', 'f0_wh6x', 'f0_wh6y', 'f0_wh6z', 'f0_wh7x', 'f0_wh7y', 'f0_wh7z', 'f0_wh8x', 'f0_wh8y', 'f0_wh8z', 'f0_wh9x', 'f0_wh9y', 'f0_wh9z', 'f0_wh10x', 'f0_wh10y', 'f0_wh10z', 'f0_wh11x', 'f0_wh11y', 'f0_wh11z', 'f0_wh12x', 'f0_wh12y', 'f0_wh12z', 'f0_wh13x', 'f0_wh13y', 'f0_wh13z', 'f0_wh14x', 'f0_wh14y', 'f0_wh14z', 'f0_wh15x', 'f0_wh15y', 'f0_wh15z', 'f0_wh16x', 'f0_wh16y', 'f0_wh16z', 'f0_wh17x', 'f0_wh17y', 'f0_wh17z', 'f0_wh18x', 'f0_wh18y', 'f0_wh18z', 'f0_wh19x', 'f0_wh19y', 'f0_wh19z', 'f0_wh20x', 'f0_wh20y', 'f0_wh20z', 'f0_cp0x', 'f0_cp0y', 'f0_cp0z', 'f0_cp11x', 'f0_cp11y', 'f0_cp11z', 'f0_cp12x', 'f0_cp12y', 'f0_cp12z'

# BERT

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertModel
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

/Users/diego/miniconda3/envs/TOG_BERT_v2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
train_config = {
  "batch_size" : 213,
  "optimizer" : optim.AdamW,
  "lr" : 2e-6,
  "loss_fn" : nn.CrossEntropyLoss,
  "num_epochs" : 10000
}

In [ ]:
# --- Prepare tensors from DataFrame ---

y = torch.tensor(df["class_numeric"].values, dtype=torch.long)
X_flat = df.drop(columns=["class_numeric"]).values
X = torch.tensor(X_flat.reshape(-1, 12, input_dim), dtype=torch.float32)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=data_config["test_ratio"], stratify=y
)

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=train_config["batch_size"], shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=train_config["batch_size"])

In [12]:
# --- Custom BERT Embeddings ---

class CustomBertEmbeddings(nn.Module):
    def __init__(self, input_dim, hidden_size, max_position_embeddings=512, dropout=0.1):
        super().__init__()
        self.linear = nn.Sequential(
            nn.Linear(input_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size)
        )
        self.position_embeddings = nn.Embedding(max_position_embeddings, hidden_size)
        self.token_type_embeddings = nn.Embedding(2, hidden_size)
        self.LayerNorm = nn.LayerNorm(hidden_size, eps=1e-12)
        self.dropout = nn.Dropout(dropout)

    def forward(
        self,
        input_ids=None,             # Needed for BERT compatibility
        token_type_ids=None,
        position_ids=None,
        inputs_embeds=None,
        past_key_values_length=0
    ):
        if inputs_embeds is None:
            raise ValueError("inputs_embeds must be provided")

        B, L, _ = inputs_embeds.size()

        if position_ids is None:
            position_ids = torch.arange(L, device=inputs_embeds.device).unsqueeze(0).expand(B, L)
        if token_type_ids is None:
            token_type_ids = torch.zeros((B, L), dtype=torch.long, device=inputs_embeds.device)

        inputs_proj = self.linear(inputs_embeds)
        pos_embed = self.position_embeddings(position_ids + past_key_values_length)
        token_type_embed = self.token_type_embeddings(token_type_ids)
        embeddings = inputs_proj + pos_embed + token_type_embed
        embeddings = self.LayerNorm(embeddings)
        return self.dropout(embeddings)

In [13]:
# --- Model ---

class BertWithCustomInput(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.bert = BertModel.from_pretrained("prajjwal1/bert-tiny")
        self.bert.embeddings = CustomBertEmbeddings(
            input_dim=input_dim,
            hidden_size=self.bert.config.hidden_size
        )
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, x):
        B, L, _ = x.shape
        position_ids = torch.arange(L, device=x.device).unsqueeze(0).expand(B, L)
        token_type_ids = torch.zeros((B, L), dtype=torch.long, device=x.device)

        outputs = self.bert(
            inputs_embeds=x,
            position_ids=position_ids,
            token_type_ids=token_type_ids
        )
        return self.classifier(outputs.pooler_output)

In [ ]:
# --- Training Setup ---

device = torch.device("mps" if torch.backends.mps.is_available()
                      else "cuda" if torch.cuda.is_available()
                      else "cpu")

model = BertWithCustomInput(input_dim=input_dim, num_classes=len(y.unique())).to(device)
optimizer = train_config["optimizer"](model.parameters(), train_config["lr"])
loss_fn = train_config["loss_fn"]()

In [ ]:
# --- Train ---

model.train()
for epoch in range(train_config["num_epochs"]):
    total_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        loss = loss_fn(logits, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss / len(train_loader):.4f}")

Epoch 1 Loss: 0.7001
Epoch 2 Loss: 0.6996
Epoch 3 Loss: 0.7033
Epoch 4 Loss: 0.7005
Epoch 5 Loss: 0.7009
Epoch 6 Loss: 0.7019
Epoch 7 Loss: 0.7081
Epoch 8 Loss: 0.6988
Epoch 9 Loss: 0.6977
Epoch 10 Loss: 0.6974
Epoch 11 Loss: 0.6908
Epoch 12 Loss: 0.7010
Epoch 13 Loss: 0.7031
Epoch 14 Loss: 0.6973
Epoch 15 Loss: 0.6977
Epoch 16 Loss: 0.7010
Epoch 17 Loss: 0.6931
Epoch 18 Loss: 0.6957
Epoch 19 Loss: 0.7000
Epoch 20 Loss: 0.6935
Epoch 21 Loss: 0.7030
Epoch 22 Loss: 0.6918
Epoch 23 Loss: 0.6955
Epoch 24 Loss: 0.6985
Epoch 25 Loss: 0.6951
Epoch 26 Loss: 0.6970
Epoch 27 Loss: 0.6969
Epoch 28 Loss: 0.6931
Epoch 29 Loss: 0.6974
Epoch 30 Loss: 0.6932
Epoch 31 Loss: 0.6998
Epoch 32 Loss: 0.6934
Epoch 33 Loss: 0.7003
Epoch 34 Loss: 0.6910
Epoch 35 Loss: 0.6963
Epoch 36 Loss: 0.6952
Epoch 37 Loss: 0.6950
Epoch 38 Loss: 0.6944
Epoch 39 Loss: 0.6937
Epoch 40 Loss: 0.6897
Epoch 41 Loss: 0.6955
Epoch 42 Loss: 0.6991
Epoch 43 Loss: 0.6929
Epoch 44 Loss: 0.6949
Epoch 45 Loss: 0.6902
Epoch 46 Loss: 0.69

In [16]:
# --- Evaluate ---

model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        logits = model(xb)
        preds = logits.argmax(dim=1).cpu()
        all_preds.append(preds)
        all_labels.append(yb)

y_pred = torch.cat(all_preds)
y_true = torch.cat(all_labels)

print("\nClassification Report:\n")
print(classification_report(y_true, y_pred))



Classification Report:

              precision    recall  f1-score   support

           0       0.94      0.73      0.82        22
           1       0.77      0.95      0.85        21

    accuracy                           0.84        43
   macro avg       0.86      0.84      0.84        43
weighted avg       0.86      0.84      0.84        43

